**incremental data ingestin**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.streaming import *

In [0]:
dbutils.widgets.text("src","")

In [0]:
src_value =  dbutils.widgets.get("src")

In [0]:
df = spark.readStream.format("cloudFiles").option("cloudFiles.format", "csv")\
    .option("cloudFiles.schemaLocation",f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoint")\
    .option("cloudFiles.schemaEvolutionMode", "rescue")\
    .load(f"/Volumes/workspace/raw/rawvolume/rawdata/{src_value}/")

In [0]:
df.writeStream.format("delta")\
    .outputMode("append")\
    .trigger(once=True)\
    .option("checkpointLocation",f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoint")\
    .option("path",f"/Volumes/workspace/bronze/bronzevolume/{src_value}/data")\
    .start()

In [0]:
%sql
select * from delta.`/Volumes/workspace/bronze/bronzevolume/bookings/data`

In [0]:
df = df.withColumn("amount",col("amount").cast("double"))\
    .withColumn("modifiedData",current_timestamp())\
    .withColumn("booking_date",to_date(col("booking_date"),"MM/dd/yyyy"))\
    .drop("_rescued_data")

#display(df)
